In [28]:
## AUTHOR: Àlex López Diaz
## Date: 20/11/2022

In [1]:
# Imports necessaris
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd
from random import randint
from time import sleep

# URLs a UTILITZAR
URL_SCRAP = 'https://marvel.fandom.com/wiki/Character_Index'
URL_SCRAP_A = 'https://marvel.fandom.com/wiki/Character_Index/B'
CSV_PATH = './Result/'
CSV_NAME = 'result_web_scrap_marvel.csv'

In [2]:
#######################################################
##                    CAS BASE                       ##
#######################################################

In [3]:
# CAS BASE
# Utilitzarem el cas base d'extraure l'informació d'una pàgina en contret
# per automatitzar-ho més endavant

page = requests.get(URL_SCRAP_A)
soup = BeautifulSoup(page.content)
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Character Index/B | Marvel Database | Fandom
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"4dfdbdb825895c7c","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Character_Index/B","wgTitle":"Character Index/B","wgCurRevisionId":8543403,"wgRevisionId":8543403,"wgArticleId":1221485,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Pages with reference errors","Characters"],"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgRelevantPageName":"Character_Index/B","wg

In [4]:
# Extraiem els fragments que contenen la informació dels personatges
characters_soup = soup.find_all("span",{"class": "mw-headline"})
print(characters_soup)

[<span class="mw-headline" id="Character_Index_-_B">Character Index - B</span>, <span class="mw-headline" id="Babe">Babe</span>, <span class="mw-headline" id="Babette">Babette</span>, <span class="mw-headline" id="Babs">Babs</span>, <span class="mw-headline" id="Babyface">Babyface</span>, <span class="mw-headline" id="Bailey">Bailey</span>, <span class="mw-headline" id="Baird">Baird</span>, <span class="mw-headline" id="Baker">Baker</span>, <span class="mw-headline" id="Balbao">Balbao</span>, <span class="mw-headline" id="Balin">Balin</span>, <span class="mw-headline" id="Ballard">Ballard</span>, <span class="mw-headline" id="Bamf">Bamf</span>, <span class="mw-headline" id="Bank">Bank</span>, <span class="mw-headline" id="Bankhead">Bankhead</span>, <span class="mw-headline" id="Bannister">Bannister</span>, <span class="mw-headline" id="Bao">Bao</span>, <span class="mw-headline" id="Barabbus">Barabbus</span>, <span class="mw-headline" id="Barb">Barb</span>, <span class="mw-headline" id=

In [5]:
# Funció que  extreu els noms dels personatges
# Al estar la info del personatge i el nom en grups separats de l'html,
# farem dos datasets amb la mateixa llargada i farem un "merge"
def extract_characters_names(soup):
    characters_soup = soup.find_all("span",{"class": "mw-headline"})
    charecters_names = []
    for character in characters_soup:
        if not character.string.startswith("Character Index",0,15) and character.string != 'References':
            charecters_names.append(character.string)
    return charecters_names

characters_names = extract_characters_names(soup)
print(characters_names)

['Babe', 'Babette', 'Babs', 'Babyface', 'Bailey', 'Baird', 'Baker', 'Balbao', 'Balin', 'Ballard', 'Bamf', 'Bank', 'Bankhead', 'Bannister', 'Bao', 'Barabbus', 'Barb', 'Barbara', 'Barenzo', 'Barkley', 'Barnaby', 'Barnacle', 'Barnes', 'Barney', 'Barry', 'Bart', 'Bartholomew', 'Basaq', 'Bashat', 'Basir', 'Batu', 'Bawka', 'Baxter', 'Baz', 'Bazil', 'Bazz', 'Bea', 'Beacher', 'Beatrice', 'Bebo', 'Beck', 'Becky', 'Beekman', 'Bell', 'Belle', 'Beltane', 'Belvedere', 'Ben', 'Benehl', 'Benetton', 'Benitez', 'Benjamin', 'Benny', 'Benta', 'Bentley', 'Benton', 'Benway', 'Bernard', 'Bernardo', 'Bernie', 'Berry', 'Bert', 'Bertha', 'Bertie', 'Bertrallo', 'Bess', 'Bessy', 'Beth', 'Betty', 'Bezukhov', 'Biker', 'Bilge', 'Bill', 'Billings', 'Billy', 'Billy Joe', 'Bilmes', 'Binkey', 'Binky', 'Binns', 'Birch', 'Bird', 'Bishop', 'Black', 'Blackburn', 'Blackie', 'Blair', 'Blake', 'Blanche', 'Blandings', 'Bloom', 'Blue Cloud', 'Bluey', 'Bo', 'Bob', 'Bobby', 'Bodi', 'Boggs', 'Boinard', 'Boleyn', 'Bolton', 'Bongert

In [6]:
# Constructor de cada linia
# Passem el nom per paràmetre per afegir-lo al registre
def extract_character_info(name,tr):
    # Hi ha registres sense foto
    if tr.find("a", {"class": "image"}) is None:
        image = ''
    else:
        image = tr.find("a", {"class": "image"})["href"]
    # Hi ha registres sense appearences
    if tr.find("a", title = True) is None:
        appearence = ''
    else:
        appearence = tr.find("a", title = True)["title"]
    # Hi ha registres sense descripcio
    if tr.find("p") is None:
        description = ''
    else:
        description = tr.find("p").text.strip()
        
    character_data = {
        'name': name,
        'image': image,
        'appearences': appearence,
        'description': description,
    }
    return character_data

# Constructor de les linies de cada bloc
# Itera tots els blocs d'informació per crear un registre per cada un
def exctract_all_character_info(name, soup):
    character_data = []
    for i in soup.select("tr"):
        character_data.append(extract_character_info(name, i))
    
    return character_data

In [7]:
# Prova per comprovar que les funcions funcionen
character_data_soup = soup.find_all("table",{"class": "article-table"})
print(exctract_all_character_info(characters_names[0],character_data_soup[3]))

[{'name': 'Babe', 'image': 'https://static.wikia.nocookie.net/marveldatabase/images/f/f6/Babyface_%28Earth-616%29_from_Amazing_Spider-Man_Vol_1_256_0001.jpg/revision/latest?cb=20210108101827', 'appearences': 'Amazing Spider-Man Vol 1 256', 'description': 'Prince and Babyface were two homeless people arguing over being able to use a cardboard box from a refrigerator to sleep in, but after seeing a figure prowling around they got scared and decided to share the crate.[4]'}]


In [8]:
# Confirmem que els dos datasets, data i names tenen la mateixa llargada
print(len(character_data_soup))
print(len(characters_names))

171
171


In [9]:
# Funció que extreu tota la informació de la pàgina web
# Crida a la funció exctract_all_character_info que crea registres per 
# cada article. Necessita un nom per afegir-lo.
def scrap_web_with_info(soup):
    character_data_soup = soup.find_all("table",{"class": "article-table"})
    characters_names = extract_characters_names(soup)
    result = []
    for i in range(0,len(character_data_soup)-1):
        result = result + exctract_all_character_info(characters_names[i], character_data_soup[i])
    return result

print(len(scrap_web_with_info(soup)))

# Cridant aquesta funció podem realitzar l'extracció del cas base.
result = scrap_web_with_info(soup)


320


In [11]:
# Funció per exportar el dataset a un csv
def export_data(result):
    header = result[0].keys()
    dir_path = os.getcwd()
    df = pd.DataFrame(result)
    df.to_csv(CSV_PATH + CSV_NAME, sep=";", header=header)

In [12]:
#######################################################
##             PROCEDIMENT PRINCIPAL                 ##
#######################################################

In [13]:
# Obtenim les dades de la web índex
page = requests.get(URL_SCRAP)
soup = BeautifulSoup(page.content)
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Character Index | Marvel Database | Fandom
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"735bef1f38279093","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Character_Index","wgTitle":"Character Index","wgCurRevisionId":8440101,"wgRevisionId":8440101,"wgArticleId":1221248,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Characters"],"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgRelevantPageName":"Character_Index","wgRelevantArticleId":1221248,"wgIsProbab

In [14]:
# Busquem el nom dels index a afegir a la url base
# Cada web que abans hem anomenat "cas base" té per URLla combinació
# de la URL base + l'identificador de l'índex
sufixes_to_add = []

for i in soup.find_all("span",{"class": "mw-headline"}):
    if i.select("a"):
        sufixes_to_add.append(i["id"])

In [15]:
print(sufixes_to_add)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'Characters_who_have_only_appeared_on_a_cover', 'Unnamed_Symbiotes', 'Earth-295', 'Earth-928', 'Earth-982', 'Earth-1226', 'Earth-1610', 'Earth-5555', 'Earth-7642', 'Earth-8311', 'Earth-9200', 'Earth-9411', 'Earth-11081', 'Earth-11542', 'Earth-20051', 'Earth-26320_Apocryphal_Characters', 'Earth-31916', 'Earth-33900', 'Earth-58163', 'Earth-85101', 'Earth-90214', 'Earth-93060', 'Earth-98121', 'Earth-120703_Apocryphal_Characters', 'Earth-148611', 'Earth-199999', 'Earth-199999_Apocryphal_Characters', 'Earth-200111', 'Earth-400005', 'Earth-TRN133', 'Earth-TRN566', 'Earth-TRN711', 'Earth-TRN907', 'Alternate_Earths_2', 'Counter-Earth_(Heroes_Reborn)', 'Spider-Man_Unlimited_Characters', 'Unknown_Characters_-_A', 'Unknown_Characters_-_B', 'Unknown_Characters_-_C', 'Unknown_Characters_-_D', 'Unknown_Characters_-_E', 'Unknown_Characters_-_F', 'Unknown_Characters_-_G', 

In [17]:
#SCRAP!!!!!!!

result_final = []
# Capem per no iterar totes les pàgines
for i in range(0,3): # Canviar el 3 per 26 si es vol l'execució completa
    # Afegim el valor recollit a la url base
    url = URL_SCRAP + '/' + sufixes_to_add[i]
    print("Scrapping page: " + URL_SCRAP + '/' + sufixes_to_add[i])
    
    i_page = requests.get(url)
    i_soup = BeautifulSoup(i_page.content)
    
    # Extraiem la informació cridant a la funció scrap_web_with_info
    result_final = result_final + scrap_web_with_info(i_soup)
    
    # Realitzem parades random entre 1 i 5 segons per evitar bloquejos
    aux = randint(1,5)
    print(f"Sleep for: {aux} seconds.")
    sleep(aux)
    
export_data(result_final)
print("TOT OK")

Scrapping page: https://marvel.fandom.com/wiki/Character_Index/A
Sleep for: 4 seconds.
Scrapping page: https://marvel.fandom.com/wiki/Character_Index/B
Sleep for: 2 seconds.
Scrapping page: https://marvel.fandom.com/wiki/Character_Index/C
Sleep for: 4 seconds.
TOT OK
